In [13]:
import pandas as pd
import plotly.express as px
import re
import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

from Project.Database import Db

In [14]:
year1_hourly, meta = Db.load_data(hourly=True, meta=True, year=1)

In [15]:
year2_hourly, meta = Db.load_data(hourly=True, meta=True, year=2)

In [16]:
plugs = meta.loc[(meta['Parameter'] == "Status_OnOff") & (meta["Measurement_Location"] == "Kitchen") & (
    meta["Unnamed: 0"].str.contains("Plug"))]["Unnamed: 0"].tolist()  #

In [17]:
consumption = meta.loc[((meta['Parameter'] == "Power_Electrical") | (meta['Parameter'] == "Power_Thermal")) & (
    meta["Description"].str.contains("power consumption" or "used"))]["Unnamed: 0"].tolist()

In [18]:
kitchenplugs = year1_hourly[["Timestamp"] + plugs].copy()
kitchenplugs["Status"] = kitchenplugs[plugs].sum(axis=1)
kitchenplugs

,Timestamp,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker,Status
0,2013-07-01 00:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-07-01 01:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-07-01 02:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-07-01 03:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-07-01 04:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
8756,2014-06-30 20:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8757,2014-06-30 21:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8758,2014-06-30 22:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8759,2014-06-30 23:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
kitchenplugs['Timestamp'] = pd.to_datetime(kitchenplugs['Timestamp'], errors="coerce", utc=True,
                                           format="%Y-%m-%d %H:%M:%S%z")

for index in kitchenplugs.index:
    kitchenplugs.loc[index, "Timestamp"] = kitchenplugs.loc[index, "Timestamp"] - datetime.timedelta(
        seconds=(int(str(year1_hourly.loc[index, "Timestamp"]).split("-")[-1][1])) * 3_600)

#Extracts hour of Timestamp.
kitchenplugs["HourOfTimestamp"] = kitchenplugs.Timestamp.dt.hour

#Replaces values to be binary 0 or 1.
for i in plugs:
    kitchenplugs.loc[kitchenplugs[i] < 0, i] = 0
    kitchenplugs.loc[kitchenplugs[i] > 0, i] = 1

kitchenplugs

,Timestamp,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker,Status,HourOfTimestamp
0,2013-07-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2013-07-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,2013-07-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2013-07-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,2013-07-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...
8756,2014-06-30 20:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
8757,2014-06-30 21:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21
8758,2014-06-30 22:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22
8759,2014-06-30 23:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23


In [20]:
#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for a selected attribute in kitchenplugs
count_series = kitchenplugs.groupby(["HourOfTimestamp", "Load_StatusPlugLoadHandMixer"]).size().copy()

new_df = count_series.to_frame(name='size').reset_index()

only_on = new_df.loc[new_df["Load_StatusPlugLoadHandMixer"] == 1]
only_on

,HourOfTimestamp,Load_StatusPlugLoadHandMixer,size
3,2,1.0,2
20,18,1.0,309
22,19,1.0,51


In [21]:
hourON = kitchenplugs.groupby(["HourOfTimestamp"])[plugs].sum()
hourON

,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker
HourOfTimestamp,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,2.0,2.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,256.0,256.0,0.0,256.0,0.0,0.0,256.0
7,0.0,0.0,0.0,0.0,0.0,0.0,13.0
8,0.0,0.0,52.0,0.0,0.0,0.0,0.0


In [22]:
#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for all attributes in kitchenplugs

dataframes = list()
for i in plugs:
    count_series = kitchenplugs.groupby(["HourOfTimestamp", i]).size()

    new_df = count_series.to_frame(name=f'{i} ').reset_index()

    only_on = new_df.loc[new_df[i] == 1].copy()
    only_on.drop(columns=[i], inplace=True)
    dataframes.append(only_on)

hourON = pd.DataFrame(list(range(0, 24)), columns=["HourOfTimestamp"])
for i in dataframes:
    hourON = hourON.merge(i, how="outer", on="HourOfTimestamp").fillna(0).sort_values(by='HourOfTimestamp',
                                                                                      ascending=True)

hourON

,HourOfTimestamp,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,2.0,2.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,256.0,256.0,0.0,256.0,0.0,0.0,256.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,13.0
8,8,0.0,0.0,52.0,0.0,0.0,0.0,0.0
9,9,101.0,101.0,102.0,101.0,0.0,0.0,101.0


In [23]:
def normalise_dataframe(df):
    '''
    Normalises values of all attributes in a dataframe, with minmax method.
    :param df:
    :type df:
    :return:
    :rtype:
    '''
    normalised_df = df.copy()

    for column in normalised_df:
        try:
            normalised_df[column] = (normalised_df[column] - normalised_df[column].min()) / (
                    normalised_df[column].max() - normalised_df[column].min())
        except:
            normalised_df[column] = normalised_df[column]

    return normalised_df

In [29]:
import plotly.graph_objects as go

#set up data to heatmap
columns = hourON.columns[1:].tolist()
hourON[columns] = normalise_dataframe(hourON[columns])
z_data = []
for i in hourON[columns]:
    data_list = hourON[i].round(2).values.tolist()
    z_data.append(data_list)

#Annotate z_values
annotations = go.Annotations()
for n, row in enumerate(z_data):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text="" if z_data[n][m] == 0 else str(z_data[n][m]),
                                         x=hourON["HourOfTimestamp"][m], y=columns[n], xref='x1', yref='y1',
                                         showarrow=False))

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=hourON["HourOfTimestamp"],
    y=columns,
    colorscale='Teal')
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,),
    font=dict(size=16),
    xaxis_nticks=30,
    annotations=annotations, )

fig.update_xaxes(title="Hour of day")
fig.update_yaxes(title="Appliance", showticklabels=True)
fig.write_html(Db.get_save_file_directory(f"Appliance_on_ratio_heatmap.html"))
fig.show()

/Users/christiannielsen/Documents/Conda_environments/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:62: DeprecationWarning:

plotly.graph_objs.Annotations is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


/Users/christiannielsen/Documents/Conda_environments/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:143: DeprecationWarning:

plotly.graph_objs.Annotation is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation




In [25]:
"""
THIS CODE IS WRONG. IT TAKES CONSUMPTION COLUMNS. IT SHOULD TAKE STATUS COLUMNS.
"""

#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
year1_hourly["Timestamp"] = pd.to_datetime(year1_hourly["Timestamp"], format="%Y-%m-%d %H:%M:%S%z",
                                           utc=True) - pd.to_timedelta(unit="h", arg=(
year1_hourly["Timestamp"].str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[1]).astype(int))

#Extracts hour of Timestamp.
year1_hourly["HourOfTimestamp"] = year1_hourly.Timestamp.dt.hour

#Replaces values to be binary 0 or 1.
for i in consumption:
    year1_hourly.loc[year1_hourly[i] < 0, i] = 0
    year1_hourly.loc[year1_hourly[i] > 0, i] = 1

#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for all attributes in year1_hours
hourON_all = year1_hourly.groupby(["HourOfTimestamp"])[consumption].sum()

#set up data to heatmap
columns = hourON_all.columns[1:].tolist()
hourON_all[columns] = normalise_dataframe(hourON_all[columns])
hourON_all["HourOfTimestamp"] = hourON_all.index
z_data = []
for i in hourON_all[columns]:
    data_list = hourON_all[i].round(2).values.tolist()
    z_data.append(data_list)

#Annotate z_values
annotations = go.Annotations()
for n, row in enumerate(z_data):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text="" if z_data[n][m] == 0 else str(z_data[n][m]),
                                         x=hourON_all["HourOfTimestamp"][m], y=columns[n], xref='x1', yref='y1',
                                         showarrow=False))

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=hourON_all["HourOfTimestamp"],
    y=columns,
    colorscale='Teal')
)

fig.update_layout(
    margin=dict(
        l=30,
        r=30,
        b=30,
        t=50, ),
    xaxis_nticks=30,
    annotations=annotations)

fig.update_xaxes(title="Hour of day")
fig.update_yaxes(title="Appliance", showticklabels=True)

fig.show()

/Users/christiannielsen/Documents/Conda_environments/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:62: DeprecationWarning:

plotly.graph_objs.Annotations is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


/Users/christiannielsen/Documents/Conda_environments/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:143: DeprecationWarning:

plotly.graph_objs.Annotation is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


